In [103]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import numpy as np
import openai
import os
from io import StringIO

In [104]:
CredsFile = "/Users/mattraghu/Documents/chatgptsheets-7c47879663f8.json"
SheetID = "1Kr8XX4_lO-W02asMQnaP3PFyS9I3FkD19EjhQhjO4yM"

SheetID = "17-kScvKokcK6gIn8GLioD7Uvzt7CRJDZTOYSmmJoJkU"

In [105]:
def setup_api():
    scope = ['https://spreadsheets.google.com/feeds',
             'https://www.googleapis.com/auth/drive']
    creds = ServiceAccountCredentials.from_json_keyfile_name(CredsFile, scope)
    client = gspread.authorize(creds)
    return client

In [106]:
def get_sheet(client, sheet_id):
    sheet = client.open_by_key(sheet_id)
    return sheet

In [107]:
client = setup_api()
sheet = get_sheet(client, SheetID)

In [108]:
#Convert the sheet to a string that can be written into a csv File
firstSheet = sheet.get_worksheet(0)
data = firstSheet.get_all_values()
headers = data.pop(0)

df = pd.DataFrame(data, columns=headers)
string_buffer = StringIO()
df.to_csv(string_buffer, index=False,header=True)
raw_text = string_buffer.getvalue()

In [109]:
# Set up the Open AI API

#Open the api key from my documents
with open(os.path.expanduser("~/Documents/OPENAI_API_KEY.txt")) as f:
    openai.api_key = f.read().strip()
    

#Set up the Organization
openai.organization = "org-9rUnSTHOfxLvlgIoNDRgEDoZ"

In [110]:
# prompt = "Add another column to the table with the name 'Full Name' and set the values to the element names"
prompt = input("Enter a prompt: ")

#Remove commas from prompt
prompt = prompt.replace(",", "")
# Set up the prompt
messages = [
    {"role": "system", "content": "You are a bot used to read spreadsheet data and adjust it based on user input. Do not include commas in your output"},
    {"role": "user", "content": "Prompt: " + prompt},
    {"role": "user", "content": "Spreadsheet: " + raw_text},
]

# Set up the completion

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages = messages,
)

outputText = response.choices[0].message.content

In [111]:
cols = outputText.split("\n")
#Now split rows by commas
rows = []
for col in cols:
    rows.append(col.split(","))
rows

[['Spreadsheet: Name', '', '', '', '', '', '', '', '', ''],
 ['Stevens Honor Code Pledge', '', '', '', '', '', '', '', '', ''],
 ['Electrified Garage Company ',
  '',
  '',
  '',
  '',
  '',
  '',
  'Note: Numbers should go in the white areas. There may be unused cells in the model.',
  '',
  ''],
 ['Income Statement', '', '', '', '', '', '', '', '', ''],
 ['"For the Year Ended December 31',
  ' 2022"',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  ''],
 ['', '', '', '', '', '', '', '', '', ''],
 ['', 'Revenues', '', ' Amount ', '', '', '', '', '', ''],
 ['Electric Car Sales', '', '', '', '', ' $18000000', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', '', '', ''],
 ['Total Revenues', '', '', '', '', ' $18000000', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', '', '', ''],
 ['', 'Expenses', '', ' Amount ', '', '', '', '', '', ''],
 ['Electric Battery Manufacturing Expense',
  '',
  '',
  '',
  '',
  ' $2318000',
  '',
  '',
  '',
  '',
  '',
  ''],
 ['Selling',
  ' 

In [112]:
#Write the output to the google sheet
outputSheet = sheet.get_worksheet(1)
outputSheet.update(rows)

{'spreadsheetId': '17-kScvKokcK6gIn8GLioD7Uvzt7CRJDZTOYSmmJoJkU',
 'updatedRange': 'Sheet1!A1:R70',
 'updatedRows': 70,
 'updatedColumns': 18,
 'updatedCells': 800}